In [104]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
clean_dataset = np.loadtxt("datasets/clean_dataset.txt")
noisy_dataset = np.loadtxt("datasets/noisy_dataset.txt")

In [96]:
class Node:
    def __init__(self, attribute, value, left, right, label = None):
        self.attribute = attribute
        self.value = value
        self.left = left
        self.right = right
        self.label = label

    def is_leaf(self):
        return not self.left and not self.right

In [97]:
def entropy(dataset):
    num = dataset.shape[0]
    labels, label_counts = np.unique(dataset, return_counts=True)
    probs = label_counts / num
    entropy = -1 * np.sum(probs * np.log2(probs))
    return entropy

In [98]:
def remainder(left, right):
    left_n = left.shape[0]
    right_n = right.shape[0]
    total_n = left_n + right_n
    rem = left_n / total_n * entropy(left) + right_n / total_n * entropy(right)
    return rem

In [99]:
def gain(total, left, right):
    return entropy(total) - remainder(left, right)

In [141]:
def find_split(dataset):
    attrs = dataset.shape[1] - 1
    h_max, attr_max, val_max = -1, None, None

    for i in range(attrs):
        points = np.unique(np.sort(dataset[:, i]))
        splits = np.sum(np.vstack((points[:-1], points[1:])), axis=0) / 2
        for val in splits:
            left = dataset[dataset[:, i] <= val]
            right = dataset[dataset[:, i] > val]
            left_labels = left[:, -1]
            right_labels = right[:, -1]
            h = gain(dataset[:, -1], left[:, -1], right[:, -1])

            print(h, i, val, left.shape, right.shape,
                  left_labels, right_labels)

            if(h > h_max):
                h_max, attr_max, val_max = h, i, val

    return (attr_max, val_max, left, right)


In [142]:
print(find_split(clean_dataset))

0.0010005414619562725 (1, 8) (1999, 8) [1.] [1. 1. 1. ... 4. 4. 4.]
0.004008685145924451 (4, 8) (1996, 8) [1. 1. 1. 1.] [1. 1. 1. ... 4. 4. 4.]
0.0070266653065993445 (7, 8) (1993, 8) [1. 1. 1. 1. 1. 1. 1.] [1. 1. 1. ... 4. 4. 4.]
0.008634248041093029 (11, 8) (1989, 8) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 4.] [1. 1. 1. ... 4. 4. 4.]
0.011750806342823772 (16, 8) (1984, 8) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 4. 4.] [1. 1. 1. ... 4. 4. 4.]
0.014636117605619337 (22, 8) (1978, 8) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 4. 4. 4. 4.] [1. 1. 1. ... 4. 4. 4.]
0.030704656542373865 (41, 8) (1959, 8) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 4. 4. 4. 4. 4.] [1. 1. 1. ... 4. 4. 4.]
0.045835967627609664 (67, 8) (1933, 8) [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.

In [102]:
def decision_tree_learning(training_set, depth = 0):
    labels = np.unique(training_set[:,-1])

    # There is only one unique label
    if(labels.shape[0] == 1):
        return Node(0,0,None,None, labels[0]), depth

    attr_max, val_max, left, right = find_split(training_set)
    
    l_branch, l_depth = decision_tree_learning(left, depth + 1)
    r_branch, r_depth = decision_tree_learning(right, depth + 1)

    node = Node(attr_max, val_max, l_branch, r_branch)

    return node, max(l_depth, r_depth)



In [120]:
dec_tree, total_depth = decision_tree_learning(clean_dataset)

In [134]:
def _draw_tree(root, depth=0):
    if(root.left == None and root.right == None):
        return ">   " + "class " + str(root.label)

    return ("| ") * depth + "feature " + str(root.attribute) + " <= " + str(root.value) + '\n' + _draw_tree(root.left, depth + 1) + '\n' + ("|   ") * depth + "feature " + str(root.attribute) + " > " + str(root.value) + '\n' + _draw_tree(root.right, depth + 1)


In [135]:
def draw_tree(tree):
    return _draw_tree(tree)

In [136]:
print(draw_tree(dec_tree))

feature 0 <= -54.5
| feature 0 <= -54.5
| | feature 0 <= -54.5
| | | feature 0 <= -54.5
| | | | feature 0 <= -54.5
| | | | | feature 0 <= -54.5
| | | | | | feature 0 <= -54.5
| | | | | | | feature 0 <= -54.5
| | | | | | | | feature 0 <= -54.5
| | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | | | | | | feature 0 <= -54.5
| | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | | | | | | feature 4 <= -56.5
| | | | | | | | | | | | | | | | | | | | | | | feature 4 <